In [1]:
# preprocess the real data from Elham lab and peprform the archetype analysis
import copy
import importlib
import os
import re
import sys
import os

parent_folder = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_folder)
import anndata
import numpy as np

import pandas as pd
import scanpy as sc

from sklearn.metrics import silhouette_score

import scvi
import seaborn as sns
from anndata import AnnData
from matplotlib import pyplot as plt
from py_pcha import PCHA
from scipy.sparse import issparse
from scipy.spatial.distance import cdist
from scipy.stats import zscore
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sympy.physics.units import current
from tqdm import tqdm
from kneed import KneeLocator

import bar_nick_utils
import covet_utils

importlib.reload(bar_nick_utils)
importlib.reload(covet_utils)
from covet_utils import compute_covet

from bar_nick_utils import preprocess_rna, preprocess_protein, plot_archetypes, \
    get_cell_representations_as_archetypes_cvxpy, reorder_rows_to_maximize_diagonal, evaluate_distance_metrics, \
    plot_archetypes_matching, compare_matchings, find_best_pair_by_row_matching, add_spatial_data_to_prot, \
    clean_uns_for_h5ad, get_latest_file

plot_flag = False
# computationally figure out which ones are best
np.random.seed(8)
! which python



/opt/conda/envs/scvi/bin/python


In [2]:
### reading in data

In [3]:

adata_1 = sc.read("../rna_umap.h5ad") # 5546 × 13447 
adata_2 = sc.read("../codex_cn_tumor.h5ad") # 893987 × 30

# filter out all tumor type cells
adata_2 = adata_2[adata_2.obs['cell_type'] != 'tumor']
adata_2 = adata_2[adata_2.obs['cell_type'] != 'dead']

num_rna_cells = 6000
num_protein_cells = 20000
# num_rna_cells = num_protein_cells= 1000
subsample_n_obs_rna = min(adata_1.shape[0],num_rna_cells)
subsample_n_obs_protein = min(adata_2.shape[0],num_protein_cells)
sc.pp.subsample(adata_1, n_obs=subsample_n_obs_rna)
sc.pp.subsample(adata_2, n_obs=subsample_n_obs_protein)

adata_1.obs['cell_types'] = adata_1.obs['new_annotation']
adata_2.obs['cell_types'] = adata_2.obs['cell_type']

# nk cells seems to mess up the archetype matching, for now remove them
adata_1 = adata_1[adata_1.obs['cell_types'] != 'nk cells'] 
adata_2 = adata_2[adata_2.obs['cell_types'] != 'nk cells']


adata_1 = adata_1[adata_1.obs['cell_types'].argsort(), :]  # sort by cell types for easier visualization
# adata_2 = adata_2[adata_2.obs['cell_types'].argsort(), :]  # sort by cell types for easier visualization
adata_2 = adata_2[adata_2.obs['cell_types'].argsort(), :]  # sort by cell types for easier visualization

# initial_adata_rna = adata[adata.obs['batch'] == f'SLN111-D1']
# initial_adata_protein = adata[adata.obs['batch'] == f'SLN208-D1'] 
# get rna from one patient, protein from the other then run the whole archetype analysis
# cell type protein, cell type rna, see which one's 
if plot_flag:
    pass  # too long
    sc.pp.pca(adata_2, n_comps=10)
    sc.pp.neighbors(adata_2)  # Compute the neighbors needed for UMAP
    sc.tl.umap(adata_2)  # Calculate UMAP coordinates
    sc.pl.umap(adata_2, color='cell_types')
    # same for rna
    sc.pp.pca(adata_1, n_comps=10)
    sc.pp.neighbors(adata_1)  # Compute the neighbors needed for UMAP
    sc.tl.umap(adata_1)  # Calculate UMAP coordinates
    sc.pl.umap(adata_1, color='cell_types')

In [4]:
# make sure we dont have gene column in var if it is equal to the index
if 'gene' in adata_1.var.columns and np.array_equal(adata_1.var['gene'].values, (adata_1.var.index.values)):
    adata_1.var.drop(columns='gene', inplace=True)
if 'gene' in adata_2.var.columns and np.array_equal(adata_2.var['gene'].values, (adata_2.var.index.values)):
    adata_2.var.drop(columns='gene', inplace=True)

In [5]:
set(adata_1.obs['new_annotation']), set(adata_2.obs['cell_type'])

({'IFN-gamma actv macs',
  'T-reg',
  'cDC-1',
  'cd206+ macs',
  'cd4+ T cells',
  'cd8+T cells',
  'cx3cr1+ macs',
  'inos+ macrophages',
  'neutrophils'},
 {'IFN-gamma actv macs',
  'T-reg',
  'cDC-1',
  'cd206+ macs',
  'cd4+ T cells',
  'cd8+T cells',
  'cx3cr1+ macs',
  'endothelial cells',
  'inos+ macrophages',
  'neutrophils'})

In [6]:
# filer out the cell types that are not in the other dataset
mutual_cell_types = set(adata_1.obs['new_annotation']).intersection(set(adata_2.obs['cell_type']))
adata_1 = adata_1[adata_1.obs['new_annotation'].isin(mutual_cell_types)]
adata_2 = adata_2[adata_2.obs['cell_type'].isin(mutual_cell_types)]
adata_1.obs['major_cell_types'] = adata_1.obs['new_annotation'].values
adata_2.obs['major_cell_types'] = adata_2.obs['cell_type'].values

/var/tmp/ipykernel_14964/506856982.py:5: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_1.obs['major_cell_types'] = adata_1.obs['new_annotation'].values
/var/tmp/ipykernel_14964/506856982.py:6: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_2.obs['major_cell_types'] = adata_2.obs['cell_type'].values


In [7]:
# Filter cells and genes (different sample)
sc.pp.filter_cells(adata_1, min_genes=100)
sc.pp.filter_genes(adata_1, min_cells=20)
# Identify highly variable genes (for further analysis, could narrow down)
sc.pp.highly_variable_genes(adata_1, n_top_genes=2000, flavor='seurat_v3')

/home/barroz/.local/lib/python3.10/site-packages/scanpy/preprocessing/_highly_variable_genes.py:75: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(


In [8]:

# Scatter plot of variance vs. mean expression
# common approach to inspect the variance of genes. It shows the relationship between mean expression and variance (or dispersion) and highlights the selected highly variable genes.
if plot_flag:
    plt.figure(figsize=(8, 6))
    plt.scatter(adata_1.var['means'], adata_1.var['variances'], alpha=0.3, label='All genes')
    plt.scatter(adata_1.var['means'][adata_1.var['highly_variable']],
                adata_1.var['variances'][adata_1.var['highly_variable']],
                color='red', label='Highly variable genes')
    plt.xlabel('Mean expression')
    plt.ylabel('Variance')
    plt.xscale('log')
    plt.yscale('log')
    plt.legend()
    plt.title('Variance vs. Mean Expression of Genes')
    plt.show()

In [9]:
# plt.figure(figsize=(8, 6))
# plt.hist(adata_1.var['variances'], bins=75, alpha=0.7)
# plt.axvline(x=adata_1.var['variances'][adata_1.var['highly_variable']].min(), color='red', linestyle='dashed', label='Cutoff')
# plt.xlabel('Variance')
# plt.ylabel('Frequency')
# plt.yscale('log')
# plt.title('Distribution of Gene Variances')
# plt.legend()
# plt.show()

In [10]:
variances_sorted = np.sort(adata_1.var['variances'])[::-1]

if plot_flag:
    plt.figure(figsize=(8, 6))
    plt.plot(range(1, len(variances_sorted) + 1), variances_sorted)
    plt.xlabel('Gene rank')
    plt.ylabel('Variance')
    # plt.xscale('log')

    plt.yscale('log')
    plt.title('Elbow plot of Gene Variances')
    plt.axvline(x=1000, color='red', linestyle='dashed', label='n_top_genes=1000')
    plt.legend()
    plt.show()
    plt.figure()
kneedle = KneeLocator(range(1, len(variances_sorted) + 1), np.log(variances_sorted), S=4.0, curve="convex",
                      direction="decreasing")
if plot_flag:
    kneedle.plot_knee()


In [11]:
print(kneedle.knee)

251


In [12]:
sc.pp.highly_variable_genes(adata_1, n_top_genes=kneedle.knee, flavor='seurat_v3')

/home/barroz/.local/lib/python3.10/site-packages/scanpy/preprocessing/_highly_variable_genes.py:75: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(


In [13]:
adata_1 = adata_1[:, adata_1.var['highly_variable']]
print(f"Selected {adata_1.shape[1]} highly variable genes.")
# PCA after selecting highly variable genes
sc.pp.pca(adata_1)
print(
    f"Variance ratio after highly variable gene selection PCA (10 PCs): {adata_1.uns['pca']['variance_ratio'][:10].sum():.4f}")



Selected 251 highly variable genes.
Variance ratio after highly variable gene selection PCA (10 PCs): 0.6337


/home/barroz/.local/lib/python3.10/site-packages/scanpy/preprocessing/_pca.py:314: ImplicitModificationWarning: Setting element `.obsm['X_pca']` of view, initializing view as actual.
  adata.obsm[key_obsm] = X_pca


In [14]:
adata_1 = adata_1[
    adata_1.obs.sort_values(by=['cell_types']).index
]

adata_2 = adata_2[
    adata_2.obs.sort_values(by=['cell_types']).index
]


In [15]:
# initial adata code (different samples) (takes a little while longer for some reason)
# make sure I do this to adata_2 as well

adata_1_rna = adata_1.copy()
sc.pp.pca(adata_1_rna)
print(f'variance explained by first 10 PCs {adata_1_rna.uns["pca"]["variance_ratio"][:10].sum()}')
adata_1_rna = preprocess_rna(adata_1_rna)
sc.pp.pca(adata_1_rna)

variance explained by first 10 PCs 0.6336579004716218
Variance ratio after log transformation PCA (10 PCs): 0.6337
Variance ratio after normalization PCA (10 PCs): 0.5848


In [16]:
# adata_1_rna.obsm.pop('protein_expression')
# assert len(set(adata.obs['batch']))!=1
adata_2_prot = adata_2
# # add all obs and var to adata_prot
# adata_prot.obs = adata_rna.obs (this is a potential problem as rna and protein obs don't match currently)


In [17]:
sc.pp.filter_cells(adata_2_prot, min_genes=30)
sc.pp.filter_genes(adata_2_prot, min_cells=50)

/home/barroz/.local/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:168: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs["n_genes"] = number


In [18]:
if plot_flag:
    sc.pp.pca(adata_2_prot)
    sc.pp.neighbors(adata_2_prot)  # Compute the neighbors needed for UMAP
    sc.tl.umap(adata_2_prot)  # Calculate UMAP coordinates
    sc.pl.umap(adata_2_prot, color='cell_types')
    # same for rna
    sc.pp.pca(adata_1_rna)
    sc.pp.neighbors(adata_1_rna)  # Compute the neighbors needed for UMAP
    sc.tl.umap(adata_1_rna)  # Calculate UMAP coordinates
    sc.pl.umap(adata_1_rna, color='cell_types')

In [19]:
adata_2_prot.obs = adata_2_prot.obs.drop(columns=['n_genes'])
adata_2_prot = preprocess_protein(adata_2_prot)
adata_2_prot.obsm.pop('X_pca')
adata_2_prot.varm.pop('PCs')
original_protein_num = adata_2_prot.X.shape[1]

Variance ratio after PCA (10 PCs): 0.8359

Variance ratio after normalization PCA (10 PCs): 0.8388

Variance ratio after log transformation PCA (10 PCs): 0.7818


In [20]:
adata_2_prot.obs['major_cell_types'].unique()
adata_2_prot.obs['cell_types'].unique()

['IFN-gamma actv macs', 'T-reg', 'cDC-1', 'cd4+ T cells', 'cd8+T cells', 'cd206+ macs', 'cx3cr1+ macs', 'inos+ macrophages', 'neutrophils']
Categories (9, object): ['IFN-gamma actv macs', 'T-reg', 'cDC-1', 'cd4+ T cells', ..., 'cd206+ macs', 'cx3cr1+ macs', 'inos+ macrophages', 'neutrophils']

In [21]:

assert adata_2_prot.obs.index.is_unique

# this is commented out because we will use the actual spatial data
# adata_2_prot,horizontal_splits,vertical_splits = add_spatial_data_to_prot(adata_2_prot, major_to_minor_dict) 
# adata_2_prot.obsm['spatial_location'] = pd.DataFrame([adata_2_prot.obs['X'],adata_2_prot.obs['Y']]).T
x_coor = adata_2_prot.obsm['spatial'][:, 0]
y_coor = adata_2_prot.obsm['spatial'][:, 1]
temp = pd.DataFrame([x_coor, y_coor], index=['x', 'y']).T
temp.index = adata_2_prot.obs.index
adata_2_prot.obsm['spatial_location'] = temp
adata_2_prot.obs['X'] = x_coor
adata_2_prot.obs['Y'] = y_coor
if plot_flag:
    sc.pl.scatter(adata_2_prot[adata_2_prot.obs['major_cell_types'] == 'T-reg'], x='X', y='Y', color='cell_types',
                  title='B Cell subtypes locations')
    # sc.pl.scatter(adata_2_prot[adata_2_prot.obs['major_cell_types']=='CD4 T'], x='X', y='Y', color='cell_types', title='T Cell subtypes locations')
    # sc.pl.scatter(adata_2_prot[adata_2_prot.obs['major_cell_types']=='CD8 T'], x='X', y='Y', color='cell_types', title='T Cell subtypes locations')

In [22]:
clean_uns_for_h5ad(adata_2_prot)
clean_uns_for_h5ad(adata_1_rna)
time_stamp = pd.Timestamp.now().strftime("%Y-%m-%d-%H-%M-%S")
adata_1_rna.write(f'data/preprocessed_adata_rna_{time_stamp}.h5ad')
adata_2_prot.write(f'data/preprocessed_adata_prot_{time_stamp}.h5ad')
